## Setting Up:

In [25]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import re
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
from scipy.spatial.distance import cdist
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GroupKFold, KFold, train_test_split, GroupShuffleSplit
from calendar import month_abbr

import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm
from oggm import cfg, utils, workflow, tasks
import logging
import geopandas as gpd
import xarray as xr

import config
import massbalancemachine as mbm

from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.xgb_helpers import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
seed_all(config.SEED)
free_up_cuda()

# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

cmap = cm.devon
color_palette_glaciers = sns.color_palette(get_cmap_hex(cmap, 15))

# For bars and lines:
color_diff_xgb = '#4d4d4d'

colors = get_cmap_hex(cm.batlow, 10)
color_xgb = colors[0]
color_tim = '#c51b7d'

color_winter = '#a6cee3'
color_annual = '#1f78b4'

# Violin and boxplots:
colors_temp_freq = sns.color_palette(get_cmap_hex(cm.devon, 8))
boxplot_style = {
    "width": .6,
    "showcaps": False,
    "palette": colors_temp_freq,
    "flierprops": {
        "marker": "x"
    },
    "showmeans": True,
    "meanprops": {
        "markerfacecolor": "white"
    }
}

custom_working_dir = '../../../data/OGGM/'

In [27]:
# RGI Ids:
# Read rgi ids:
path_rgi = '../../../data/GLAMOS/CH_glacier_ids_long.csv'
rgi_df = pd.read_csv(path_rgi, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)

In [28]:
# Specify the short names of the climate variables available in the dataset
vois_climate = [
    't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]

vois_topographical = [
    "aspect",
    "slope",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]

## Read GL data:

In [29]:
# Read stakes data over all glaciers:
data_glamos = pd.read_csv(
    path_PMB_GLAMOS_csv +
    f'CH_wgms_dataset_all.csv')  # with snow probes too (not only stakes)

print('Number of glaciers:', len(data_glamos['GLACIER'].unique()))

# Capitalize glacier names:
glacierCap = {}
for gl in data_glamos['GLACIER'].unique():
    if gl == 'claridenU':
        glacierCap[gl] = 'Clariden_U'
    elif gl == 'claridenL':
        glacierCap[gl] = 'Clariden_L'
    else:
        glacierCap[gl] = gl.capitalize()

# Glaciers with data of potential clear sky radiation
# Format to same names as stakes:
glDirect = [
    re.search(r'xr_direct_(.*?)\.nc', f).group(1)
    for f in os.listdir(path_direct_save)
]
glDirect.sort()

restgl = Diff(list(glDirect), list(data_glamos.GLACIER.unique()))
restgl.sort()

# Filter out glaciers without data:
data_glamos = data_glamos[data_glamos.GLACIER.isin(glDirect)]

Number of glaciers: 43


## Input data:
### Input dataset:

In [30]:
RUN = False
if RUN:
    # Filter data_glamos
    print('Running on {} glaciers:\n {}'.format(
        len(data_glamos.GLACIER.unique()), data_glamos.GLACIER.unique()))
    # Create dataloader:
    dataset_gl = mbm.Dataset(data=data_glamos,
                             region_name='CH',
                             data_path=path_PMB_GLAMOS_csv)
    print('Number of winter and annual samples:', len(data_glamos))
    print('Number of annual samples:',
          len(data_glamos[data_glamos.PERIOD == 'annual']))
    print('Number of winter samples:',
          len(data_glamos[data_glamos.PERIOD == 'winter']))

    # Add climate data:
    # Specify the files of the climate data, that will be matched with the coordinates of the stake data
    era5_climate_data = path_ERA5_raw + 'era5_monthly_averaged_data.nc'
    geopotential_data = path_ERA5_raw + 'era5_geopotential_pressure.nc'

    # Match the climate features, from the ERA5Land netCDF file, for each of the stake measurement dataset
    dataset_gl.get_climate_features(climate_data=era5_climate_data,
                                    geopotential_data=geopotential_data,
                                    change_units=True)

    # Add potential clear sky radiation:
    print('----------------------\nAdding potential clear sky radiation:')
    print('Shape before pot rad:', dataset_gl.data.shape)
    dataset_gl.get_potential_rad(path_direct_save)
    print('Shape after pot rad:', dataset_gl.data.shape)

    print('----------------------\nConverting to monthly resolution:')
    # For each record, convert to a monthly time resolution
    dataset_gl.convert_to_monthly(meta_data_columns=config.META_DATA,
                                  vois_climate=vois_climate + ['pcsr'],
                                  vois_topographical=vois_topographical)

    # Create a new DataLoader object with the monthly stake data measurements.
    dataloader_gl = mbm.DataLoader(data=dataset_gl.data,
                                   random_seed=config.SEED,
                                   meta_data_columns=config.META_DATA)

    print('Number of monthly rows:', len(dataloader_gl.data))
    print('Columns in the dataset:', dataloader_gl.data.columns)

    # save the data
    dataloader_gl.data.to_csv(path_PMB_GLAMOS_csv +
                              'CH_wgms_dataset_monthly_full.csv',
                              index=False)
else:
    # read data
    data_monthly = pd.read_csv(path_PMB_GLAMOS_csv +
                               'CH_wgms_dataset_monthly_full.csv')
    dataloader_gl = mbm.DataLoader(data=data_monthly,
                                   random_seed=config.SEED,
                                   meta_data_columns=config.META_DATA)
    print('Number of monthly rows:', len(dataloader_gl.data))
    print('Number of annual rows:',
          len(dataloader_gl.data[dataloader_gl.data.PERIOD == 'annual']))
    print('Number of winter rows:',
          len(dataloader_gl.data[dataloader_gl.data.PERIOD == 'winter']))

Number of monthly rows: 300905
Number of annual rows: 102252
Number of winter rows: 198653


### Correction for elevation difference:

In [31]:
path_glogem_factors = os.path.join(path_glogem, 'reference_run_GloGEM2024.csv')
glogem_factors = pd.read_csv(path_glogem_factors)
glogem_factors = glogem_factors.rename(columns={'ID': 'RGIId'})

glogem_factors['RGIId'] = glogem_factors['RGIId'].apply(
    lambda x: format_rgi_code(x))

# Correct t2m and tp for elevation
data_monthly = correctTP(
    data_monthly,
    gl_specific=False,
    c_prec=2.3,  # Precipitation correction factor
    t_off=2.03  # Temperature offset
)

# Save with corrected values:
dataloader_gl = mbm.DataLoader(data=data_monthly,
                               random_seed=config.SEED,
                               meta_data_columns=config.META_DATA)

In [32]:
c_prec_dic, t_off_dic = {}, {}
for gl in data_monthly.GLACIER.unique():
    rgi_gl = data_monthly[data_monthly.GLACIER == gl].RGIId.unique()[0]
    factor_gl = glogem_factors[glogem_factors.RGIId == rgi_gl]
    c_prec_dic[gl] = factor_gl['Cprec'].values[0]
    t_off_dic[gl] = factor_gl['T_off'].values[0]

# mean of dic values
mean_c_prec = np.min(list(c_prec_dic.values()))
mean_t_off = np.min(list(t_off_dic.values()))

# print min and max of dic values
print('Min c_prec:', np.min(list(c_prec_dic.values())), ', Max c_prec:',
      np.max(list(c_prec_dic.values())))
print('Min t_off:', np.min(list(t_off_dic.values())), ', Max t_off:',
      np.max(list(t_off_dic.values())))


Min c_prec: 1.3 , Max c_prec: 2.3
Min t_off: -0.02 , Max t_off: 2.03


## Blocking on glaciers:

In [33]:
test_glaciers = [
    'tortin', 'plattalva', 'sanktanna', 'schwarzberg', 'hohlaub', 'pizol',
    'joeri', 'corvatsch', 'tsanfleuron'
]
train_glaciers = [
    i for i in dataloader_gl.data.GLACIER.unique() if i not in test_glaciers
]

data_test = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(test_glaciers)]
print('Size of test data:', len(data_test))
data_train = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
    train_glaciers)]
print('Size of train data:', len(data_train))

test_perc = (len(data_test) / len(data_train)) * 100
print('Percentage of test size: {:.2f}%'.format(test_perc))

# CV splits
splits, test_set, train_set = getCVSplits(dataloader_gl,
                                          test_split_on='GLACIER',
                                          test_splits=test_glaciers)

print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))
test_perc = (len(test_set['df_X']) / len(train_set['df_X'])) * 100
print('Percentage of test size: {:.2f}%'.format(test_perc))
print('Size of test set:', len(test_set['df_X']))
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Size of train set:', len(train_set['df_X']))

Size of test data: 32974
Size of train data: 267931
Percentage of test size: 12.31%
Test glaciers: (9) ['corvatsch' 'hohlaub' 'joeri' 'pizol' 'plattalva' 'sanktanna'
 'schwarzberg' 'tortin' 'tsanfleuron']
Percentage of test size: 12.31%
Size of test set: 32974
Train glaciers: (25) ['adler' 'albigna' 'aletsch' 'allalin' 'basodino' 'claridenL' 'claridenU'
 'corbassiere' 'findelen' 'forno' 'gietro' 'gorner' 'gries' 'limmern'
 'morteratsch' 'murtel' 'oberaar' 'otemma' 'pers' 'plainemorte' 'rhone'
 'schwarzbach' 'sexrouge' 'silvretta' 'taelliboden']
Size of train set: 267931


## XGBoost:

In [34]:
# Grid search
param_grid = {
    'max_depth': [2, 3, 4, 5, 6, 7, 8],
    'n_estimators':
    [50, 100, 200, 300, 400, 500, 600,
     700],  # number of trees (too many = overfitting, too few = underfitting)
    'learning_rate': [0.01, 0.1, 0.15, 0.2, 0.25, 0.3]
}

param_init = {}
param_init['device'] = 'cuda:0'
param_init['tree_method'] = 'hist'
param_init["random_state"] = config.SEED
param_init["n_jobs"] = config.NUM_JOBS

vois_climate = [
    't2m_corr', 'tp_corr', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]

# Feature columns:
feature_columns = [
    'ELEVATION_DIFFERENCE'
] + list(vois_climate) + list(vois_topographical) + ['pcsr']
all_columns = feature_columns + config.META_DATA + config.NOT_METADATA_NOT_FEATURES
df_X_train_subset = train_set['df_X'][all_columns]
print('Shape of training dataset:', df_X_train_subset.shape)
print('Shape of testing dataset:', test_set['df_X'][all_columns].shape)
print('Running with features:', feature_columns)

Shape of training dataset: (267931, 27)
Shape of testing dataset: (32974, 27)
Running with features: ['ELEVATION_DIFFERENCE', 't2m_corr', 'tp_corr', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10', 'aspect', 'slope', 'hugonnet_dhdt', 'consensus_ice_thickness', 'millan_v', 'pcsr']


In [ ]:
c_prec_range = np.linspace(1, 4, 30)
#t_off_range = np.linspace(-1, 2, 10)
t_off_range = [0]
custom_params = {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 800}

satellite_years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
satellite_glaciers = [
    'adler', 'aletsch', 'allalin', 'basodino', 'claridenL', 'claridenU',
    'findelen', 'gries', 'hohlaub', 'limmern', 'oberaar', 'plattalva', 'rhone',
    'sanktanna', 'schwarzbach', 'schwarzberg'
]

scores = []
c_prec_, t_off_ = [], []

RUN = True
if RUN:
    emptyfolder('results/nc/sensitivity/')

    # first delete results file
    if os.path.exists(f'results/sensitivity_scores.csv'):
        os.remove(f'results/sensitivity_scores.csv')

    # Add header
    with open(f'results/sensitivity_scores.csv', 'w') as f:
        f.write("c_prec,t_off,score\n")

    # for c_prec, t_off in tqdm(zip(np.random.choice(c_prec_range, size=10),
    #                             np.random.choice(t_off_range, size=10)),
    #                         desc='c_prec and t_off',
    #                         position=0):
    for c_prec in tqdm(c_prec_range, desc='c_prec', position=0):
        c_prec_.append(c_prec)
        t_off = 0.0
        t_off_.append(t_off)

        # Correct t2m and tp for elevation
        data_monthly = correctTP(
            data_monthly,
            gl_specific=False,
            c_prec=c_prec,  # Precipitation correction factor
            t_off=t_off  # Temperature offset
        )

        dataloader_gl = mbm.DataLoader(data=data_monthly,
                                       random_seed=config.SEED,
                                       meta_data_columns=config.META_DATA)

        data_test = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
            test_glaciers)]
        data_train = dataloader_gl.data[dataloader_gl.data.GLACIER.isin(
            train_glaciers)]

        test_perc = (len(data_test) / len(data_train)) * 100

        # CV splits
        splits, test_set, train_set = getCVSplits(dataloader_gl,
                                                  test_split_on='GLACIER',
                                                  test_splits=test_glaciers)

        df_X_train_subset = train_set['df_X'][all_columns]
        params = {**param_init, **custom_params}
        custom_model = mbm.models.CustomXGBoostRegressor(**params)

        # Fit on train data:
        custom_model.fit(train_set['df_X'][all_columns], train_set['y'])

        # Make predictions on test
        custom_model = custom_model.set_params(device='cpu')

        # Calculate scores
        score = custom_model.score(test_set['df_X'][all_columns],
                                   test_set['y'])  # negative
        scores.append(score)

        # write score to file
        with open('results/sensitivity_scores.csv', 'a') as f:
            f.write(f'{c_prec},{t_off},{score}\n')

        for glacierName in tqdm(satellite_glaciers,
                                desc='Glaciers',
                                position=1,
                                leave=False):
            # Get stake measurements for the glacier
            rgi_gl = rgi_df.loc[glacierName]['rgi_id.v6']
            data_gl = data_glamos[data_glamos.RGIId == rgi_gl]
            dataset_gl = mbm.Dataset(data=data_gl,
                                     region_name='CH',
                                     data_path=path_PMB_GLAMOS_csv)
            # Get OGGM glacier mask and grid
            ds, glacier_indices, gdir = dataset_gl.get_glacier_mask(
                custom_working_dir)

            # Get glacier grid (preprocessed):
            df_grid_monthly = pd.read_csv(path_glacier_grid +
                                          f'{glacierName}_grid.csv')
            df_grid_monthly['GLACIER'] = glacierName
            df_grid_monthly['POINT_ELEVATION'] = df_grid_monthly['topo']
            df_grid_monthly.drop_duplicates(inplace=True)  # remove duplicates

            df_grid_monthly = correctTP(
                df_grid_monthly)  # Correct T & P for altitude

            df_grid_monthly = df_grid_monthly[all_columns]

            # Add cumulative monthly SMB:
            df_grid_monthly = cumulativeMonthly(df_grid_monthly, custom_model)
            dataloader = mbm.DataLoader(data=df_grid_monthly,
                                        meta_data_columns=config.META_DATA)

            # Seasonal predictions:
            grouped_ids_annual = GlacierWidePred(custom_model,
                                                 df_grid_monthly[all_columns],
                                                 type_pred='annual')

            grouped_ids_winter = GlacierWidePred(custom_model,
                                                 df_grid_monthly[all_columns],
                                                 type_pred='winter')

            # Save to netcdf for mapping:
            path_lv95 = f"results/nc/sensitivity/{glacierName}/lv95/"  # Swiss coordinate system
            path_wgs84 = f"results/nc/sensitivity/{glacierName}/wgs84/"  # Laton coordinate system

            for year in tqdm(satellite_years,
                             desc='Years',
                             leave=False,
                             position=2):
                # Annual SMB:
                pred_y_annual = grouped_ids_annual[grouped_ids_annual.YEAR ==
                                                   year].drop(['YEAR'], axis=1)

                ds_pred_annual_latlon, ds_pred_annual_xy = predXarray(
                    ds, gdir, pred_y_annual)
                save_to_netcdf(ds_pred_annual_latlon, path_wgs84,
                               f"{glacierName}_{year}_annual.nc")
                save_to_netcdf(ds_pred_annual_xy, path_lv95,
                               f"{glacierName}_{year}_annual.nc")

                # Winter SMB:
                pred_y_winter = grouped_ids_winter[grouped_ids_winter.YEAR ==
                                                   year].drop(['YEAR'], axis=1)
                ds_pred_winter_latlon, ds_pred_winter_xy = predXarray(
                    ds, gdir, pred_y_winter)
                save_to_netcdf(ds_pred_winter_latlon, path_wgs84,
                               f"{glacierName}_{year}_winter.nc")
                save_to_netcdf(ds_pred_winter_xy, path_lv95,
                               f"{glacierName}_{year}_winter.nc")

                # Save monthly grids:
                df_grid_monthly_gl_y = df_grid_monthly[
                    (df_grid_monthly['GLACIER'] == glacierName)
                    & (df_grid_monthly['YEAR'] == year)]

                for i, month in enumerate(month_abbr_hydr.keys()):
                    if month == 'sep_':
                        continue
                    monthNb = month_abbr_hydr[month]
                    df_grid_gl_m = df_grid_monthly_gl_y[
                        df_grid_monthly_gl_y['MONTHS'] == month]

                    # Get in format for map:
                    df_grid_gl_m = df_grid_gl_m.groupby('ID').agg({
                        'YEAR':
                        'mean',
                        'POINT_LAT':
                        'mean',
                        'POINT_LON':
                        'mean',
                        'pred':
                        'mean',
                        'cum_pred':
                        'mean'
                    })
                    pred_y = df_grid_gl_m.drop(['YEAR'], axis=1)
                    ds_pred_latlon, ds_pred_xy = predXarray(
                        ds, gdir, pred_y, pred_var='cum_pred')
                    # save grids
                    save_to_netcdf(
                        ds_pred_xy, path_lv95,
                        f"{glacierName}_{year}_{monthNb}_{np.round(c_prec, 4)}_{np.round(t_off, 4)}.nc"
                    )
                    save_to_netcdf(
                        ds_pred_latlon, path_wgs84,
                        f"{glacierName}_{year}_{monthNb}_{np.round(c_prec, 4)}_{np.round(t_off, 4)}.nc"
                    )
        free_up_cuda()

c_prec:   0%|          | 0/30 [00:00<?, ?it/s]

Glaciers:   0%|          | 0/16 [00:00<?, ?it/s]

2024-12-16 13:28:27: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-12-16 13:28:27: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-12-16 13:28:27: oggm.cfg: Multiprocessing: using all available processors (N=32)
2024-12-16 13:28:27: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2024-12-16 13:28:27: oggm.cfg: Multiprocessing switched ON after user settings.
2024-12-16 13:28:27: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2024-12-16 13:28:27: oggm.workflow: init_glacier_directories from prepro level 3 on 1 glaciers.
2024-12-16 13:28:27: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers
2024-12-16 13:28:28: oggm.workflow: Execute entity tasks [gridded_attributes] on 1 glaciers


Years:   0%|          | 0/7 [00:00<?, ?it/s]

2024-12-16 13:28:58: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-12-16 13:28:58: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-12-16 13:28:58: oggm.cfg: Multiprocessing: using all available processors (N=32)
2024-12-16 13:28:58: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2024-12-16 13:28:58: oggm.cfg: Multiprocessing switched ON after user settings.
2024-12-16 13:28:58: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2024-12-16 13:28:58: oggm.workflow: init_glacier_directories from prepro level 3 on 1 glaciers.
2024-12-16 13:28:58: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers
2024-12-16 13:28:58: oggm.workflow: Execute entity tasks [gridded_attributes] on 1 glaciers


...

Glaciers:   0%|          | 0/16 [00:00<?, ?it/s]

2024-12-16 13:56:47: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-12-16 13:56:47: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-12-16 13:56:47: oggm.cfg: Multiprocessing: using all available processors (N=32)
2024-12-16 13:56:47: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2024-12-16 13:56:47: oggm.cfg: Multiprocessing switched ON after user settings.
2024-12-16 13:56:47: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2024-12-16 13:56:47: oggm.workflow: init_glacier_directories from prepro level 3 on 1 glaciers.
2024-12-16 13:56:47: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers
2024-12-16 13:56:47: oggm.workflow: Execute entity tasks [gridded_attributes] on 1 glaciers


Years:   0%|          | 0/7 [00:00<?, ?it/s]

2024-12-16 13:57:17: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-12-16 13:57:17: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-12-16 13:57:17: oggm.cfg: Multiprocessing: using all available processors (N=32)
2024-12-16 13:57:17: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2024-12-16 13:57:17: oggm.cfg: Multiprocessing switched ON after user settings.
2024-12-16 13:57:17: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2024-12-16 13:57:17: oggm.workflow: init_glacier_directories from prepro level 3 on 1 glaciers.
2024-12-16 13:57:17: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers
2024-12-16 13:57:17: oggm.workflow: Execute entity tasks [gridded_attributes] on 1 glaciers


Years:   0%|          | 0/7 [00:00<?, ?it/s]